Research question 1: Do concept-based explanations produce more faithful explanations than feature attribution methods?

In [1]:
pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Step 0:Setup

In [129]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array

blackbox_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

def preprocess_images(img_array):
    img_array = np.array([tf.image.resize(img_to_array(img), (224, 224)) for img in img_array])
    return preprocess_input(img_array)

def black_box_classify(img_array,convert_to_nr=True):
    preprocessed_imgs = preprocess_images(img_array)
    predictions = blackbox_model.predict(preprocessed_imgs)
    prediction_labels = decode_predictions(predictions, top = 1)
    labels_as_str = [row[0][1] for row in prediction_labels]
    if convert_to_nr:
        label_as_nr = label_encoder.transform(labels_as_str)
        return [[l]for l in label_as_nr]
    return [[l]for l in labels_as_str]

def black_box_lime(temp):
    resized_temp = resize(temp, (224, 224), mode='reflect', preserve_range=True).astype(np.uint8)
    resized_temp = np.expand_dims(resized_temp, axis=0)
    predictions = blackbox_model.predict(resized_temp)
    prediction_labels = decode_predictions(predictions, top = 1)
    labels_as_str = [row[0][1] for row in prediction_labels]
    label_as_nr = label_encoder.transform(labels_as_str)
    return [[l]for l in label_as_nr]

In [69]:
import numpy as np
import pickle
import hashlib
import pandas as pd

np.random.seed(42)

base_path = "/Users/karlgustav/Documents/GitHub/study/master-thesis/server/src/research/"
# base_path = "/Users/karl-gustav.kallasmaa/Documents/Projects/master-thesis/server/src/"
all_labels_path = f"{base_path}all_classes.txt"
masks_path = f"{base_path}data/masks.pkl"
img_path = f"{base_path}data/resized_imgs.pkl"
labels_path = f"{base_path}data/classes.pkl"
ade_path = f"{base_path}data/objectInfo150.csv"

ade_classes = pd.read_csv(ade_path)

labels = []
images = []
masks = []
unique_labels = []
with open(masks_path, 'rb') as f:
    masks = pickle.load(f)
with open(img_path, 'rb') as f:
    images = pickle.load(f)
with open(all_labels_path) as f:
    lines = f.read().splitlines()
    lines = [l.replace(' ', '_') for l in lines]
    unique_labels = np.array(list(set(lines)))
    
all_concept_values = ade_classes['Name'].tolist()
UNIQUE_CONCEPT_VALUES = sorted(list(set(all_concept_values)))
NR_OF_UNIQUE_CONCEPTS = len(UNIQUE_CONCEPT_VALUES)


image_hex_index_map = {hashlib.sha1(np.array(img).view(np.uint8)).hexdigest(): i for i,img in enumerate(images)}

index_img_map = {i:img for i,img in enumerate(images)}
#index_label_map = {i:label for i,label in enumerate(labels)}
index_mask_map = {i:mask for i,mask in enumerate(masks)}
index_ade_map = {i:ade for i,ade in enumerate(ade_classes)}

test_size = 0.01 # 1%

random_indexes = np.random.choice(list(index_img_map.keys()), int(test_size*len(index_img_map.keys())), replace=False)

random_images = [index_img_map[index] for index in random_indexes]
#random_labels = np.array([index_label_map[index] for index in random_indexes])
random_masks = [index_mask_map[index] for index in random_indexes]
random_labels = black_box_classify(random_images,False)

1/1 [==============================] - 1s 666ms/step


In [3]:
from typing import List
from sklearn import preprocessing

def encode_categorical_values(values: List[str]):
    unique_values = sorted(list(set(values)))
    le = preprocessing.LabelEncoder()
    le.fit(unique_values)
    return le
label_encoder = encode_categorical_values(unique_labels)

Step 1: Get lime predictions

In [ ]:
from lime import lime_image,lime_tabular
from skimage.transform import resize


def explain_with_lime_OLD(images,labels):
    explainer = lime_image.LimeImageExplainer()
    explanations = []
    for i,image in enumerate(images):
        explanation = explainer.explain_instance(np.array(image),
                                                 labels=[labels[i]],
                                                 classifier_fn=black_box_classify,
                                                 top_labels=3,
                                                 batch_size=100,
                                                 num_samples=100,
                                                 hide_color=None)
        #print(explanation)
        print(explanation.top_labels)
        most_probable_label = explanation.top_labels[0]
        explanations.append(most_probable_label)
    return explanations

def explain_with_lime_tab(images,labels):
    img_arr = [np.array(image) for image in images]
    explainer = lime_tabular.LimeTabularExplainer(training_data=img_arr,feature_names=["1","2","3"])
    explanations = []
    return explanations


def explain_with_lime(images,labels):
    update_labels = [l[0] for l in labels]
    explainer = lime_image.LimeImageExplainer()
    explanations = []
    for i,image in enumerate(images):
        lime_img_exp = explainer.explain_instance(np.array(image),
                                               #  labels=update_labels,
                                                 classifier_fn=black_box_classify,
                                                 top_labels=3,
                                                 batch_size=100,
                                                 num_samples=150,
                                                 hide_color=None)
        
        temp, mask = lime_img_exp.get_image_and_mask(lime_img_exp.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
        prediction_on_temp = black_box_lime(temp)
        explanations.append(prediction_on_temp[0])
    return explanations

lime_predictions = explain_with_lime(random_images[0:2],random_labels[0:2])

In [131]:
print(lime_predictions)

[[[819]], [[568]]]


Step 2: Get concept-based desision tree explanations

In [7]:
from operator import itemgetter
from typing import Dict, List

def get_segments(img, mask, threshold=0.05):
    segs = np.unique(mask)
    segments = []
    total = mask.shape[0] * mask.shape[1]
    segments_classes = []
    for seg in segs:
        idxs = mask == seg
        sz = np.sum(idxs)
        if sz < threshold * total:
            continue
        segment = img * idxs[..., None]
        w, h, _ = np.nonzero(segment)
        segment = segment[np.min(w):np.max(w), np.min(h):np.max(h), :]
        segments.append(segment)
        segments_classes.append(ade_classes['Name'].loc[ade_classes['Idx'] == seg].iloc[0])
    return segments, segments_classes

def sort_dictionary(source: Dict[any, any], by_value=True, reverse=True) -> List[any]:
    if by_value:
        return sorted(source.items(), key=itemgetter(1), reverse=reverse)
    return sorted(source.items(), key=itemgetter(0), reverse=reverse)

In [8]:
from typing import Dict, List
from mpire import WorkerPool
from functools import reduce

class MostPopularConcepts:
    BATCH_SIZE = 10
    MAX_WORKER_COUNT = 8

    def __init__(self,l_labels,i_images,m_maks):
        all_labels = np.array(l_labels)
        chunk_size = max(1, int(all_labels.size / self.BATCH_SIZE))
        self.labels_in_chunks = np.array_split(all_labels, chunk_size)
        self.nr_of_jobs = min(self.MAX_WORKER_COUNT, len(self.labels_in_chunks))

        self.label_images_map = {}
        self.label_masks_map = {}

        self.image_most_popular_concepts = self.static_most_popular_concepts(l_labels,i_images,m_maks)

    def static_most_popular_concepts(self,l_labels,i_images,m_maks) -> Dict[str, List[any]]:
        for label, image, mask in zip(l_labels,i_images,m_maks):
            current_images = self.label_images_map.get(label, [])
            current_maks = self.label_masks_map.get(label, [])

            current_images.append(image)
            current_maks.append(mask)

            self.label_images_map[label] = current_images
            self.label_masks_map[label] = current_maks

        with WorkerPool(n_jobs=self.nr_of_jobs) as pool:
            return reduce(lambda a, b: {**a, **b},
                          pool.map(self.__extract_most_popular_concepts, self.labels_in_chunks))

    def __extract_most_popular_concepts(self, l_labels: List[str]) -> Dict[str, List[any]]:
        partial_results = {}
        for label in  l_labels:
            i_images = self.label_images_map[label]
            m_masks = self.label_masks_map[label]
            nr_of_images = len(i_images)
            partial_results[label] = self.most_popular_concepts(images,m_masks, nr_of_images)
        return partial_results

    @staticmethod
    def most_popular_concepts(i_images, m_masks, k) -> List[str]:
        segment_count = {}
        for pic, mask in zip(i_images, m_masks):
            try:
                _, seg_class = get_segments(np.array(pic), mask, threshold=0.005)
                for s in seg_class:
                    segment_count[s] = segment_count.get(s, 0) + 1
            except:
                continue
        segment_count = sort_dictionary(segment_count)
        return [s for s, _ in segment_count[:k]]

In [9]:
MOST_POPULAR_CONCEPTS = MostPopularConcepts(labels,images,masks).image_most_popular_concepts

In [10]:
from typing import List,Tuple
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


def get_segment_relative_size(segment: np.array, picture: np.array) -> float:
    segment_area = float(segment.shape[0] * segment.shape[1])
    picture_area = float(picture.shape[0] * picture.shape[1])
    return round(segment_area / picture_area, 2)


def get_training_row(top_concepts_for_label: List[str], pic, mask) -> np.array:
    row = np.zeros(NR_OF_UNIQUE_CONCEPTS)
    pic_as_array = np.array(pic)
    segss, seg_class = get_segments(pic_as_array, mask, threshold=0.005)
    for index,concept in enumerate(UNIQUE_CONCEPT_VALUES):
        if concept in top_concepts_for_label and concept in seg_class:
            segment = segss[seg_class.index(concept)]
            row[index] = get_segment_relative_size(segment, pic_as_array)            
    return row

def train_decision_tree(x, y) -> DecisionTreeClassifier:
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    return clf

In [11]:
def train_concept_explainer(all_labels,all_images,all_masks):
    X, y = [], []
    for label, pic, mask in zip(all_labels,all_images, all_masks):
        most_popular_concepts_for_label = MOST_POPULAR_CONCEPTS[label]
        row = get_training_row(most_popular_concepts_for_label, pic, mask)
        label_as_nr = label_encoder.transform([label])
        X.append(row)
        y.append(label_as_nr[0])
    return train_decision_tree(X,np.array(y))

In [12]:
def explain_with_concepts(images,model):
    predictions = []
    for img in images:
        img_key = hashlib.sha1(np.array(img).view(np.uint8)).hexdigest()
        image_index = image_hex_index_map[img_key]
        image_label = index_label_map[image_index]

        most_popular_concepts_for_label = MOST_POPULAR_CONCEPTS[image_label]
        mask = index_mask_map[image_index]
        
        row = get_training_row(most_popular_concepts_for_label, img, mask)
        prediction_as_nr = model.predict([row])
        # prediction_as_label = label_encoder.inverse_transform(prediction_as_nr)
        predictions.append(prediction_as_nr[0])
    return predictions

concept_model = train_concept_explainer(labels,images,masks)
concept_predictions = explain_with_concepts(random_images,concept_model)

Step 3: calculate fidelity

In [101]:
"""
def black_box_models_predictions(image_array):
    preprocessed_images = np.array([preprocess_image(i) for i in image_array])
    batch_prediction = blackbox_model.predict(preprocessed_images)
    prediction_label = decode_predictions(batch_prediction, top=1)
    prediction_labels = [p[0][1] for p in prediction_label]
    predictions_as_nr = label_encoder.transform(prediction_labels)
    return np.array(predictions_as_nr)
"""

def fidelity(pred1,pred2):
    same = 0
    not_same = 0
    for p1 in pred1:
        for p2 in pred2:
            if p1 == p2:
                same += 1
            else:
                not_same += 1
    return same / not_same


black_box_pred = black_box_classify(random_images)
black_box_pred = [p[0] for p in black_box_pred]

lime_fidelity = fidelity(pred1=lime_predictions,pred2=black_box_pred)
concept_fidelity = fidelity(pred1=concept_predictions,pred2=black_box_pred)

print("blackbox pred")
print(black_box_pred)
print("lime pred")
print(lime_predictions)
print("concept pred")
print(concept_predictions)


print("LIME fidelity "+str(lime_fidelity))
print("Concept fidelity "+str(concept_fidelity))
if lime_fidelity > concept_fidelity:
    diff = lime_fidelity - concept_fidelity
    print("LIME fidelity is greater than concept fidelity by "+str(diff))
else:
    diff = concept_fidelity -lime_fidelity
    print("Concept fidelity is creater than LIME fidelity by "+str(diff))

1/1 [==============================] - 0s 328ms/step
blackbox pred
[819, 892, 934, 776, 934, 754, 639, 114, 298, 754, 730, 553, 158, 566, 355]
lime pred
[0, 0]
concept pred
[179, 657, 506, 330, 270, 186, 788, 124, 270, 186, 422, 489, 143, 662, 763]
LIME fidelity 0.0
Concept fidelity 0.0
Concept fidelity is creater than LIME fidelity by 0.0
